In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from numpy import recfromtxt, column_stack, array
from pandas import DataFrame

# Import Statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
import statsmodels.api as sm
from statsmodels.tsa.base import  datetools
from statsmodels.datasets.utils import Dataset
from os.path import dirname, abspath
 
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import mlflow
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns  # to plot the heat maps

In [ ]:
# Reading Data
df = pd.read_excel("data/raw/favardata1105.xlsx")
# dfdate = df
df = df.set_index('date')
df

In [ ]:
# Taking Log of selected columns
dflog = np.log(df[[ 'ABCPI', 'ARY', 'ASI', 'BLAG', 'BLMF', 'BLOG', 'BLPS', 'BLSM', 'BLTL', 'BLUS', 'BLXP', 'C1CPI', 'C2CPI', 
                    'CCPI', 'CCPS', 'CFCPI', 'CGRY', 'COP', 'CPD', 'CPS', 'COS', 'CRY', 'ECPI', 'ER', 'EUR', 'EXR', 'FCPI',
                    'FHCPI', 'FNCPI', 'FRY', 'GBP', 'GRV', 'GXP', 'HHCPI', 'HRY', 'HWCPI', 'IEP', 'IIP', 'IMAP', 'IMIP',
                    'IMP', 'IRY', 'M1', 'M2', 'MCPI', 'MRY', 'NDC', 'NFA', 'NORY', 'PRY', 'QM', 'RCCPI', 'RHCPI', 'RINV', 
                    'RPC', 'RPDI', 'RR', 'RRY', 'RUCPI', 'RY', 'SD', 'SMRY', 'SRY', 'TCPI', 'TD', 'TRY', 'URCPI', 'URY', 
                    'USD', 'EXP', 'MPMIS']])
dflog

In [ ]:
# Diffferencing Logged Values of selected columns
dflogdiff = dflog[[ 'C1CPI','C2CPI','FCPI','FNCPI','ABCPI','CFCPI','HWCPI','FHCPI','HHCPI','TCPI','CCPI','RCCPI','ECPI',
                   'RHCPI','MCPI','URCPI','RUCPI','CPD','EXP','IMP','GXP','IRY','SMRY','MRY','SRY','TRY','CRY','URY','PRY',
                   'RINV','IMAP','IMIP','IEP','IIP','COP','GBP','EUR','ASI','M1','QM','CCPS','COS','SD','TD','RR','ER','NFA',
                   'NDC','EXR','BLPS','BLAG','BLSM','BLXP','BLMF','BLUS','BLOG','BLTL','USD' ]]

In [ ]:
dflogdiff = dflogdiff - dflogdiff.shift(4)
dflogdiff

In [ ]:
# Differencing of selected values 
dfonlydiff = df[['MDR1','MDR3','MDR6','MDR12','PLR','MLR','IBCR','TBR','CRR']]
dfonlydiff = dfonlydiff - dfonlydiff.shift(4)
dfonlydiff

In [ ]:
# GROWTH RATE of Selected Columns
grdfn = df[['HCPI','RY','ARY','ICRY','TTRY','RRY','ERY','HRY','M2','CGRY','NORY']]
growth_rate = ((grdfn - grdfn.shift(4)) / grdfn.shift(4)) * 100
growth_rate

#### Grouping of Dataframes

In [ ]:
groupL = dflog[[ 'ABCPI', 'GRV', 'RPC', 'RPDI']]
# groupL = groupL.assign(df['date'])
groupL

In [ ]:
#Note: Removed IRY from here
groupDL = dflogdiff[[ 'C1CPI','C2CPI','FCPI','FNCPI','CFCPI','HWCPI','FHCPI','HHCPI','TCPI','CCPI','RCCPI','ECPI','RHCPI',
                     'MCPI','URCPI','RUCPI','CPD','EXP','IMP','GXP','RINV','IMAP','IMIP','IEP','IIP','GBP','EUR','ASI',
                     'QM','CCPS','COS','SD','TD','ER','NFA','NDC','EXR','BLTL' ]]
groupDL

In [ ]:
#Note: Removed RY from here
groupGR = growth_rate[['HCPI', 'M2','CGRY', 'NORY']]
groupGR

In [ ]:
SubGroup = pd.merge(pd.merge(groupL, groupDL, on=['date']), pd.merge(dfonlydiff, groupGR, on=['date']), on=['date'])

In [ ]:
# Final Group
FinalGroup = pd.merge(SubGroup, df['SDR'],  on=['date'])

In [ ]:
# Dropping all rows with NaN
FinalGroup = FinalGroup.dropna()
FinalGroup

In [ ]:
# Generating Princioal Components
pca5 = PCA(n_components=5)
principalComponents = pca5.fit_transform(FinalGroup)

In [ ]:
# Converting PCs to Dataframe
Dfpc = pd.DataFrame(data = principalComponents, columns = ['pc1', 'pc2', 'pc3', 'pc4', 'pc5'], index=FinalGroup.index)
Dfpc

#### Testing for stationarity

In [ ]:
def test_stationary(timeseries):
    
    #Determining rolling statistics
    movingAvg = timeseries.rolling(window=4).mean()
    movingSTD = timeseries.rolling(window=4).std()
    
    #Plot rolling statistics:
    orig = plt.plot(timeseries, color='blue', label='Original')
    mean = plt.plot(movingAvg, color='red', label='Rolling mean')
    std = plt.plot(movingSTD, color='black', label='Rolling std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)
    
    #Perform Dickey-Fuller Test
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries.dropna(),autolag='AIC') # .dropna() handles differenced data
    labels = ['ADF test statistic','p-value','# lags used','# observations']
    out = pd.Series(dftest[0:4],index=labels)
    for key,val in dftest[4].items():
        out[f'critical value ({key})']=val
    print(out.to_string())          # .to_string() removes the line "dtype: float64"
    if dftest[1] <= 0.05:
        print("Strong evidence against the null hypothesis")
        print("Reject the null hypothesis")
        print("Data has no unit root and is stationary")
    else:
        print("Weak evidence against the null hypothesis")
        print("Fail to reject the null hypothesis")
        print("Data has a unit root and is non-stationary")

In [ ]:
test_stationary(Dfpc['pc1'])

In [ ]:
test_stationary(Dfpc['pc2'])

In [ ]:
test_stationary(Dfpc['pc3'])

In [ ]:
test_stationary(Dfpc['pc4'])

In [ ]:
test_stationary(Dfpc['pc5'])

In [ ]:
# Differencing the non stationary PCs to achieve stationarity
DfpcDiff = Dfpc - Dfpc.shift()
plt.plot(DfpcDiff)
plt.show()

In [ ]:
DfpcDiff.dropna(inplace=True)

In [ ]:
test_stationary(DfpcDiff['pc3'])

In [ ]:
# Create a dataframe with PCs and dlRY
df_sub = growth_rate["RY"].iloc[3:]
df_sub

In [ ]:
df_pcsRY = DfpcDiff.assign(dlRY=df_sub)
df_pcsRY

In [ ]:
# Sampling of Endogenous Variables
endog = df_pcsRY.loc['2017-03-31':'2021-12-31', ['pc1','pc2','pc3','pc4','pc5','dlRY']]
endog

In [ ]:
exog = np.log(df[['COP50','COP55','COP60','COP65','COP70','COP75','COP80','COP85','COP90','COP95','COP100','COP105','COP110','MPMIS1']])

log_gxp = np.log(df['GXP1'])
dfgxp = log_gxp - log_gxp.shift(4)

exog = pd.merge(exog, dfgxp,  on=['date']) 
exog

In [ ]:
# Sampling of Exogenous Variables
exogC = exog.loc['2017-03-31':'2021-12-31', ['COP50', 'GXP1', 'MPMIS1']]
exogC

## VARx Model

In [ ]:
var = VAR(endog, exogC)

In [ ]:
# Checking best lag value
x= var.select_order()
x.summary()

In [ ]:
results = var.fit(1)
#We can check the summary of the model by.
results.summary()

In [ ]:
# prob column (p-value) and only consider regressors where p-values are small (<.05)
# COP & L1.pc4
# Get the values on the cooefficient column of the two = 0.109589 & 0.324013
### dlRYts = 0.11*dlRY(COP) - 0.32*dlRY(l1.pc4)

In [ ]:
results.plot();

In [ ]:
results.save("favarmodel.pickle")

import pickle
with open('favar_learned_model.pkl','wb') as f:
    pickle.dump(results,f)

In [ ]:
import pickle

# # Open the file in binary mode
with open('favarmodel.pickle', 'rb') as file:
      
#     # Call load method to deserialze
    myvar = pickle.load(file)
    print(myvar)

In [ ]:
exogF = exog.loc['2022-03-31':'2022-12-31', ['COP50', 'GXP1', 'MPMIS1']]
exogF

In [ ]:
exogF.to_csv("exogF.csv")

In [ ]:
# df[-4:-3].index

In [ ]:
forecast = myvar.forecast(endog.values[-1:], steps=4, exog_future=exogF)

In [ ]:
forecastdf = pd.DataFrame(data = forecast, columns = ['pc1', 'pc2', 'pc3', 'pc4', 'pc5', 'dlRY'], index=exogF.index)

# forecastdf.reset_index(inplace=True)

forecastdf['dlRY']

In [ ]:
# forecastdf.set_index(exogF.index, inplace = True)

In [ ]:
growth_rate['RY'].plot()

In [ ]:
forecastdf['dlRY'].plot()

In [ ]:
exogC_55 = exog.loc['2017-03-31 00:00:00':'2021-12-31 00:00:00', ['COP55', 'GXP1', 'MPMIS1']]

# VAR 55
var55 = VAR(endog, exogC_55)
# x= var.select_order()
# x.summary()
results55 = var.fit(1)
#We can check the summary of the model by.
results55.summary()
exogF_55 = exog.loc['2022-03-31 00:00:00':'2022-12-31 00:00:00', ['COP55', 'GXP1', 'MPMIS1']]
forecast55 = results55.forecast(endog.values[-1:], steps=4, exog_future=exogF_55)
forecastdf55 = pd.DataFrame(data = forecast55, columns = ['pc1', 'pc2', 'pc3', 'pc4', 'pc5', 'dlRY'], index=exogF_55.index)
forecastdf55.reset_index(inplace=True)
forecastdf55['dlRY']

In [ ]:
forecastdf55['dlRY'].plot()

In [ ]:
exogC_60 = exog.loc['2017-03-31 00:00:00':'2021-12-31 00:00:00', ['COP60', 'GXP1', 'MPMIS1']]

# VAR 60
var60 = VAR(endog, exogC_60)
# x= var.select_order()
# x.summary()
results60 = var.fit(1)
#We can check the summary of the model by.
results60.summary()
exogF_60 = exog.loc['2022-03-31 00:00:00':'2022-12-31 00:00:00', ['COP60', 'GXP1', 'MPMIS1']]
forecast60 = results60.forecast(endog.values[-1:], steps=4, exog_future=exogF_60)
forecastdf60 = pd.DataFrame(data = forecast60, columns = ['pc1', 'pc2', 'pc3', 'pc4', 'pc5', 'dlRY'], index=exogF_60.index)
forecastdf60.reset_index(inplace=True)
forecastdf60['dlRY']

In [ ]:
forecastdf60['dlRY'].plot()

In [ ]:
exogC_65 = exog.loc['2017-03-31 00:00:00':'2021-12-31 00:00:00', ['COP65', 'GXP1', 'MPMIS1']]

# VAR 65
var65 = VAR(endog, exogC_65)
# x= var.select_order()
# x.summary()
results65 = var.fit(1)
#We can check the summary of the model by.
results65.summary()
exogF_65 = exog.loc['2022-03-31 00:00:00':'2022-12-31 00:00:00', ['COP65', 'GXP1', 'MPMIS1']]
forecast65 = results65.forecast(endog.values[-1:], steps=4, exog_future=exogF_65)
forecastdf65 = pd.DataFrame(data = forecast65, columns = ['pc1', 'pc2', 'pc3', 'pc4', 'pc5', 'dlRY'], index=exogF_65.index)
forecastdf65.reset_index(inplace=True)
forecastdf65

In [ ]:
forecastdf65['dlRY'].plot()

In [ ]:
exogC_85 = exog.loc['2017-03-31':'2021-12-31', ['COP85', 'GXP1', 'MPMIS1']]

# VAR 85
var85 = VAR(endog, exogC_85)
# x= var.select_order()
# x.summary()
results85 = var.fit(1)
#We can check the summary of the model by.
results85.summary()
exogF_85 = exog.loc['2022-03-31':'2022-12-31', ['COP85', 'GXP1', 'MPMIS1']]
forecast85 = results85.forecast(endog.values[-1:], steps=4, exog_future=exogF_85)
forecastdf85 = pd.DataFrame(data = forecast85, columns = ['pc1', 'pc2', 'pc3', 'pc4', 'pc5', 'dlRY'], index=exogF_85.index)
forecastdf85.reset_index(inplace=True)
forecastdf85['dlRY']

In [ ]:
exogC_105 = exog.loc['2017-03-31 00:00:00':'2021-12-31 00:00:00', ['COP105', 'GXP1', 'MPMIS1']]

# VAR 105
var105 = VAR(endog, exogC_105)
# x= var.select_order()
# x.summary()
results105 = var.fit(1)
#We can check the summary of the model by.
results105.summary()
exogF_105 = exog.loc['2022-03-31 00:00:00':'2022-12-31 00:00:00', ['COP105', 'GXP1', 'MPMIS1']]
forecast105 = results105.forecast(endog.values[-1:], steps=4, exog_future=exogF_105)
forecastdf105 = pd.DataFrame(data = forecast105, columns = ['pc1', 'pc2', 'pc3', 'pc4', 'pc5', 'dlRY'], index=exogF_105.index)
forecastdf105.reset_index(inplace=True)
forecastdf105['dlRY']

In [ ]:
forecastdf105['dlRY'].plot()

In [ ]:
# forecast105r["realgdp_forecasted"] = growth_rate['RY'].iloc[-10-1] + forecastdf105['dlRY'.cumsum()

In [ ]:
# growth_rate['RY'].plot()

In [ ]:
exogC_110 = exog.loc['2017-03-31 00:00:00':'2021-12-31 00:00:00', ['COP110', 'GXP1', 'MPMIS1']]

# VAR 110
var110 = VAR(endog, exogC_110)
# x= var.select_order()
# x.summary()
results110 = var.fit(1)
#We can check the summary of the model by.
results110.summary()
exogF_110 = exog.loc['2022-03-31 00:00:00':'2022-12-31 00:00:00', ['COP110', 'GXP1', 'MPMIS1']]
forecast110 = results110.forecast(endog.values[-1:], steps=4, exog_future=exogF_110)
forecastdf110 = pd.DataFrame(data = forecast110, columns = ['pc1', 'pc2', 'pc3', 'pc4', 'pc5', 'dlRY'], index=exogF_110.index)
forecastdf110.reset_index(inplace=True)
forecastdf110['dlRY']

In [ ]:
forecastdf110['dlRY'].plot()

#### Author: Bello Ahmed Dangiwa
#### Central Bank of Nigeria
